In [27]:

# coding: utf-8

# In[118]:

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D, AtrousConvolution1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from seya.layers.recurrent import Bidirectional # after git clone need to python setup.py install again
# from keras.utils.layer_utils import print_layer_shapes

print 'loading data'
# trainmat = h5py.File('/Users/jx/Downloads/deepsea_train/train.mat')
validmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/pcavalid.mat')
testmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/pcatest.mat')
trainmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/pcatrain.mat')

X_train = np.expand_dims(np.array(trainmat['tr'][0][0][0]),axis=(1))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train.shape



lr = 1e-6#learning rate
reg = 1e-4
print 'building model'
nb_filters=32
model = Sequential()
# model.add(LSTM(32,  W_regularizer=l2(reg),return_sequences=True, input_shape=(4, 113)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(64,  W_regularizer=l2(reg),return_sequences=True)) # return sequences is needed for stacking
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(128,  W_regularizer=l2(reg)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

model.add(Convolution1D(64, 29, border_mode='same', W_regularizer=l2(reg),input_shape=(1, 47))) # adding conv layer collapses output
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Convolution1D(32, 4, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Convolution1D(32, 4, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('tanh'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(1))
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


print 'compiling model'
model.compile(loss='mse', optimizer='adam', metrics=["mse"])



checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

X_valid=np.expand_dims(validmat['v'][0][0][0],axis=(1))
y_valid=np.array(validmat['v'][0][0][1]).squeeze()


model.fit(X_train, y_train, batch_size=256, nb_epoch=500, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])





loading data
(23428, 1, 47)
(23428,)
building model
compiling model
Train on 23428 samples, validate on 2929 samples
Epoch 1/500
23428/23428 [==============================] - 2s - loss: 0.0575 - mean_squared_error: 0.0574 - val_loss: 0.0552 - val_mean_squared_error: 0.0552
Epoch 2/500
23428/23428 [==============================] - 2s - loss: 0.0536 - mean_squared_error: 0.0535 - val_loss: 0.0551 - val_mean_squared_error: 0.0551
Epoch 3/500
23428/23428 [==============================] - 2s - loss: 0.0536 - mean_squared_error: 0.0535 - val_loss: 0.0549 - val_mean_squared_error: 0.0549
Epoch 4/500
23428/23428 [==============================] - 2s - loss: 0.0536 - mean_squared_error: 0.0535 - val_loss: 0.0547 - val_mean_squared_error: 0.0547
Epoch 5/500
23428/23428 [==============================] - 2s - loss: 0.0537 - mean_squared_error: 0.0536 - val_loss: 0.0549 - val_mean_squared_error: 0.0549
Epoch 6/500
23428/23428 [==============================] - 3s - loss: 0.0536 - mean_squared_e

In [28]:
%matplotlib

out=model.predict(X_train, batch_size=512,verbose=1)
plt.plot(y_train,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
23428/23428 [==============================] - 4s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_28 (Convolution1D) (None, 1, 64)         87296       convolution1d_input_9[0][0]      
____________________________________________________________________________________________________
activation_25 (Activation)       (None, 1, 64)         0           convolution1d_28[0][0]           
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 1, 64)         0           activation_25[0][0]              
____________________________________________________________________________________________________
convolution1d_29 (Convolution1D) (None, 1, 32)         8224        dropout_25[0][0]                 
___

In [29]:
%matplotlib
X_test=np.expand_dims(testmat['tt'][0][0][0],axis=(1))
y_test=np.array(testmat['tt'][0][0][1]).squeeze()

out=model.predict(X_test, batch_size=512,verbose=1)
plt.plot(y_test,out,'ro')

model.summary()

Using matplotlib backend: MacOSX
2928/2928 [==============================] - 0s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_28 (Convolution1D) (None, 1, 64)         87296       convolution1d_input_9[0][0]      
____________________________________________________________________________________________________
activation_25 (Activation)       (None, 1, 64)         0           convolution1d_28[0][0]           
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 1, 64)         0           activation_25[0][0]              
____________________________________________________________________________________________________
convolution1d_29 (Convolution1D) (None, 1, 32)         8224        dropout_25[0][0]                 
_____